# Create the train the model

The XGBoost model will is created from the preprocessed Nettiauto dataset.

In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option('display.max_columns', None)


## Load preprocessed dataset

The dataset file has been created in the: `nettiauto_dataset_parsing.ipynb`

In [2]:
df = pd.read_csv('./nettiauto_dataset.csv')
# df = df.drop('color', axis=1)

df.head(25)

,make,model,color,driveType,price,totalOwners,kilometers,seats,power,batteryCapacity,age,isSuv,metallicColor,airconditioning,parking_sensors,satellite_navigator,cruise_control_adaptive,parking_camera_simple_camera,seat_heaters,driving_assistant,alloy_wheels,electric_mirrors,lane_departure_warning_system,electrically_operated_tailgate,leather_upholstery,emergency_brake_assist,collision_avoidance_system,cruise_control_traditional,parking_assistant,tow_bar,electric_seats_with_memory,heated_steering_wheel,sunroof,sun_hatch_with_panorama,adaptive_headlights,sport_seats,fuel_battery_powered_heater,parking_camera_360-degree_camera,battery_preheating,electric_seats_without_memory,air_suspension,curve_lights,head_up_display,sport_base
0,porsche,taycan,black,rwd,101880.0,1,2100.0,5,300.0,79.00,2.416667,0,0,1,1,1,0,0,1,0,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
1,mercedes-benz,eqs,gray,4wd,125900.0,1,30000.0,5,385.0,108.40,2.000000,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,1,1,1,1,1,0,1,0,0,0
2,nissan,leaf,black,fwd,35800.0,1,10000.0,5,160.0,62.00,2.000000,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,citroen,e-c4,blue,fwd,33800.0,1,17000.0,5,100.0,50.00,2.000000,0,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,hyundai,kona,white,fwd,27590.0,1,56000.0,5,150.0,64.00,2.750000,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,1,0
5,citroen,e-c4,beige,fwd,29700.0,1,34000.0,5,100.0,50.00,2.000000,0,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,polestar,2,black,4wd,60780.0,1,7000.0,5,300.0,75.00,2.000000,0,0,1,1,1,1,0,1,1,1,1,0,1,0,1,1,0,1,0,1,1,1,0,1,1,1,1,0,0,0,1,0,0
7,peugeot,e-2008,orange,fwd,25790.0,1,13000.0,5,100.0,50.00,2.500000,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,bmw,ix3,white,rwd,51800.0,1,17000.0,5,210.0,80.00,2.166667,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
9,bmw,ix3,white,rwd,49900.0,1,25500.0,5,210.0,80.00,2.000000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0


## Boolean conversion

- Accessories 0/1 values (present/not-present), such as metallicColor = 1 or 0
- New columns are created for different *Make, model, color* and *driveType* strings and exclusive boolean values given for each

In [3]:
# Change Accessories 0/1 integer values to booleans
df.iloc[:, 11:] = df.iloc[:, 11:].astype(bool)

# Convert to make, model, color and dataType strings to distinct boolean values
df_encoded = pd.get_dummies(df, columns=[
    "make",
    "model",
    "color",
    "driveType"
])

df_encoded.head(25)


,price,totalOwners,kilometers,seats,power,batteryCapacity,age,isSuv,metallicColor,airconditioning,parking_sensors,satellite_navigator,cruise_control_adaptive,parking_camera_simple_camera,seat_heaters,driving_assistant,alloy_wheels,electric_mirrors,lane_departure_warning_system,electrically_operated_tailgate,leather_upholstery,emergency_brake_assist,collision_avoidance_system,cruise_control_traditional,parking_assistant,tow_bar,electric_seats_with_memory,heated_steering_wheel,sunroof,sun_hatch_with_panorama,adaptive_headlights,sport_seats,fuel_battery_powered_heater,parking_camera_360-degree_camera,battery_preheating,electric_seats_without_memory,air_suspension,curve_lights,head_up_display,sport_base,make_audi,make_bmw,make_byd,make_citroen,make_cupra,make_dacia,make_ds,make_fiat,make_ford,make_hyundai,make_jaguar,make_kia,make_lexus,make_maxus,make_mazda,make_mercedes-benz,make_mg,make_mini,make_muu merkki,make_nissan,make_opel,make_peugeot,make_polestar,make_porsche,make_renault,make_seat,make_skoda,make_subaru,make_tesla,make_toyota,make_volkswagen,make_volvo,model_2,model_208,model_3,model_500e,model_ariya,model_atto3,model_born,model_bz4x,model_c4,model_c40,model_citigo,model_combo-e,model_cooper,model_corsa-e,model_e-2008,model_e-208,model_e-308,model_e-berlingo,model_e-c4,model_e-c4 x,model_e-niro,model_e-soul,model_e-spacetourer,model_e-traveller,model_e-tron,model_e-tron gt,model_e-tron gt rs,model_e-up!,model_enyaq,model_eqa,model_eqb,model_eqc,model_eqe,model_eqs,model_eqs suv,model_eqv,model_euniq,model_ev6,model_ev9,model_han,model_i-pace,model_i3,model_i3s,model_i4,model_i4 m50,model_i7,model_id. buzz,model_id3,model_id4,model_id5,model_ioniq 5,model_ioniq 6,model_ioniq electric,model_ix,model_ix m60,model_ix1,model_ix3,model_kona,model_kona electric,model_leaf,model_marvel r,model_megane,model_mg4,model_mg5,model_mifa 9,model_mii electric,model_model 3,model_model s,model_model x,model_model y,model_mokka,model_mokka-e,model_mustang,model_mustang mach-e,model_muu malli,model_mx-30,model_niro,model_proace city verso ev,model_proace verso ev,model_q4 e-tron,model_q8 e-tron,model_solterra,model_spring,model_t90,model_tang,model_taycan,model_twingo,model_up!,model_ux,model_xc40,model_zafira-e,model_zoe,model_zs,color_beige,color_black,color_blue,color_brown,color_copper,color_gray,color_green,color_orange,color_other,color_red,color_silver,color_turquoise,color_violet,color_white,color_yellow,driveType_4wd,driveType_fwd,driveType_rwd
0,101880.0,1,2100.0,5,300.0,79.00,2.416667,False,False,True,True,True,False,False,True,False,True,True,False,True,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,125900.0,1,30000.0,5,385.0,108.40,2.000000,False,False,True,True,True,True,False,True,True,True,True,True,True,True,False,True,False,True,False,True,False,False,True,True,True,True,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

## Model training

### Test split

In [ ]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop('price', axis=1)
y = df_encoded['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape

### XGBoost fit

In [5]:
import xgboost as xgb

xgb_regressor = xgb.XGBRegressor(objective ='reg:squarederror',
                                 n_estimators=100,
                                 max_depth=8,
                                 reg_lambda=1, # näitä voi säädellä muttei kauheesti oo vaikuttanut malliin
                                 reg_alpha=0,# näitä voi säädellä muttei kauheesti oo vaikuttanut malliin
                                 num_parallel_tree=5, # random forest 🤷‍♂️
)

xgb_regressor.fit(X_train, y_train)

/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_ca

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=5, random_state=None, ...)

## Test prediction

In [6]:
y_pred = xgb_regressor.predict(X_test)


/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_ca

### The mean absolute error

In [7]:
mean_absolute_error(y_test, y_pred)

3857.0957178470476

### The mean squared error

In [8]:
mean_squared_error(y_test, y_pred)

45842891.59066879

### R² score

In [9]:
r2_score(y_test, y_pred)

0.9393566946977366

## Test predictions by make and model

In [10]:
def predict_make_model(make, model):
    m = (X_test[f'make_{make}'] == 1) & (X_test[f'model_{model}'] == 1)
    X_id3 = X_test[m]
    y_id3 = y_test[m]

    y_id3_pred = xgb_regressor.predict(X_id3)

    print('mean_absolute_error: {}'.format(mean_absolute_error(y_id3, y_id3_pred)))
    print('r2_score: {}'.format(r2_score(y_id3, y_id3_pred)))
    print()
    for real, pred in zip(y_id3, y_id3_pred):
        print(f'{real} -> {pred} ~ {real - pred} <> {pred / real}')

mean_absolute_error: 2425.6958295036766
r2_score: 0.8415075116132087

28890.0 -> 31717.994140625 ~ -2827.994140625 <> 1.0978883399316373
50565.0 -> 45171.7109375 ~ 5393.2890625 <> 0.8933394825966577
28400.0 -> 28770.552734375 ~ -370.552734375 <> 1.0130476314920775
29380.0 -> 28746.3671875 ~ 633.6328125 <> 0.9784331922226004
36900.0 -> 33307.6875 ~ 3592.3125 <> 0.9026473577235772
49700.0 -> 46523.9375 ~ 3176.0625 <> 0.9360953219315895
31950.0 -> 30822.263671875 ~ 1127.736328125 <> 0.9647030883215962
41890.0 -> 40510.28515625 ~ 1379.71484375 <> 0.9670633840116973
41900.0 -> 44968.15625 ~ -3068.15625 <> 1.073225686157518
29900.0 -> 27799.8984375 ~ 2100.1015625 <> 0.9297624895484949
47000.0 -> 45567.66796875 ~ 1432.33203125 <> 0.9695248503989362
42340.0 -> 44020.28515625 ~ -1680.28515625 <> 1.0396855256554085
29890.0 -> 30593.29296875 ~ -703.29296875 <> 1.0235293733271997
49610.0 -> 45567.66796875 ~ 4042.33203125 <> 0.9185177982009676
46500.0 -> 44398.8984375 ~ 2101.1015625 <> 0.9548150201

/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_ca

### Wolkswagen ID3

In [ ]:
predict_make_model('volkswagen', 'id3')

### BMW ix1

In [11]:
predict_make_model('bmw', 'ix1')

mean_absolute_error: 3838.725360576923
r2_score: -0.6094595606164213

72238.0 -> 67815.5390625 ~ 4422.4609375 <> 0.9387792998491099
67270.0 -> 68056.1640625 ~ -786.1640625 <> 1.0116866963356623
61500.0 -> 57756.8671875 ~ 3743.1328125 <> 0.9391360518292683
72900.0 -> 71340.640625 ~ 1559.359375 <> 0.9786096107681755
69880.0 -> 67944.1953125 ~ 1935.8046875 <> 0.9722981584502003
71580.0 -> 74297.4921875 ~ -2717.4921875 <> 1.0379644060841018
69990.0 -> 75053.7421875 ~ -5063.7421875 <> 1.072349509751393
66050.0 -> 67888.546875 ~ -1838.546875 <> 1.0278356831945497
78290.0 -> 65519.3828125 ~ 12770.6171875 <> 0.8368806081555754
68900.0 -> 65906.65625 ~ 2993.34375 <> 0.9565552431059506
69900.0 -> 65816.75 ~ 4083.25 <> 0.9415844062947067
65790.0 -> 58755.578125 ~ 7034.421875 <> 0.8930776428788569
69830.0 -> 70785.09375 ~ -955.09375 <> 1.0136774130030073


/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_ca

### Tesla 3

In [12]:
predict_make_model('tesla', 'model 3')

mean_absolute_error: 2201.733708079268
r2_score: 0.6554887976828042

43950.0 -> 52526.33984375 ~ -8576.33984375 <> 1.1951385629977247
42990.0 -> 47497.8046875 ~ -4507.8046875 <> 1.1048570525122121
38500.0 -> 38584.09375 ~ -84.09375 <> 1.0021842532467533
45890.0 -> 45659.89453125 ~ 230.10546875 <> 0.9949857165232077
50790.0 -> 48306.65234375 ~ 2483.34765625 <> 0.9511055787310494
34990.0 -> 32825.328125 ~ 2164.671875 <> 0.9381345563018005
43780.0 -> 46301.61328125 ~ -2521.61328125 <> 1.0575973796539515
48950.0 -> 43741.3984375 ~ 5208.6015625 <> 0.8935934307967314
49790.0 -> 48293.13671875 ~ 1496.86328125 <> 0.9699364675386624
48470.0 -> 48012.1796875 ~ 457.8203125 <> 0.9905545633897256
37900.0 -> 37098.0 ~ 802.0 <> 0.9788390501319261
48490.0 -> 49357.35546875 ~ -867.35546875 <> 1.0178873060167044
36370.0 -> 37152.62890625 ~ -782.62890625 <> 1.0215185291792686
41890.0 -> 40142.6875 ~ 1747.3125 <> 0.9582880759131057
45690.0 -> 43721.7890625 ~ 1968.2109375 <> 0.9569225008207485
37880.0 -> 3

/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_ca

### Opel Mokka-e

In [13]:
predict_make_model('opel', 'mokka-e')

mean_absolute_error: 1877.900390625
r2_score: 0.25110915901931896

32990.0 -> 36908.078125 ~ -3918.078125 <> 1.1187656297362838
37780.0 -> 38830.6015625 ~ -1050.6015625 <> 1.0278084055717311
38470.0 -> 39962.59375 ~ -1492.59375 <> 1.0387989017416168
40770.0 -> 34200.88671875 ~ 6569.11328125 <> 0.8388738464250675
41660.0 -> 40682.59375 ~ 977.40625 <> 0.9765384961593855
38570.0 -> 40268.921875 ~ -1698.921875 <> 1.0440477540834845
40900.0 -> 40996.36328125 ~ -96.36328125 <> 1.0023560704462102
40970.0 -> 39379.0 ~ 1591.0 <> 0.9611667073468392
39390.0 -> 35702.3359375 ~ 3687.6640625 <> 0.9063807041761869
32500.0 -> 31886.376953125 ~ 613.623046875 <> 0.9811192908653846
37700.0 -> 39306.4609375 ~ -1606.4609375 <> 1.0426116959549072
41570.0 -> 41007.37890625 ~ 562.62109375 <> 0.9864656941604523
41800.0 -> 42348.2578125 ~ -548.2578125 <> 1.013116215610048


/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_ca

### Audi e-tron

In [14]:
predict_make_model('audi', 'e-tron')

mean_absolute_error: 8995.14697265625
r2_score: -0.9491781381402378

47900.0 -> 56973.9375 ~ -9073.9375 <> 1.1894350208768267
64999.0 -> 68135.03125 ~ -3136.03125 <> 1.0482473768827212
78790.0 -> 69363.0234375 ~ 9426.9765625 <> 0.8803531341223505
50800.0 -> 50843.87890625 ~ -43.87890625 <> 1.0008637579970472
49900.0 -> 50862.47265625 ~ -962.47265625 <> 1.0192880291833668
67900.0 -> 65624.09375 ~ 2275.90625 <> 0.966481498527246
68900.0 -> 75025.0625 ~ -6125.0625 <> 1.0888978592162555
63000.0 -> 107873.3125 ~ -44873.3125 <> 1.7122748015873015
49800.0 -> 53675.734375 ~ -3875.734375 <> 1.0778259914658634
64900.0 -> 52373.68359375 ~ 12526.31640625 <> 0.8069905022149461
56800.0 -> 59407.27734375 ~ -2607.27734375 <> 1.0459027701364436
48780.0 -> 48620.36328125 ~ 159.63671875 <> 0.9967274145397704
52800.0 -> 93307.4765625 ~ -40507.4765625 <> 1.7671870561079546
75800.0 -> 70586.2421875 ~ 5213.7578125 <> 0.9312169154023747
86900.0 -> 89204.859375 ~ -2304.859375 <> 1.0265231228423475
61900.0 -> 6

/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/jatuja/.local/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_ca